In [13]:
from aco import ACO
import numpy as np
import logging
from gen_inst import BPPInstance, load_dataset, dataset_conf

N_ANTS = 30
N_ITERATIONS = [1, 10, 30, 50, 80, 100]


def heuristics_reevo(demand: np.ndarray, capacity: int) -> np.ndarray:
    n = demand.shape[0]
    demand_normalized = demand / demand.max()
    
    same_bin_penalty = np.abs((capacity - demand[:, None] - demand) / capacity)
    overlap_penalty = (demand[:, None] + demand) / capacity
    
    heuristics = demand_normalized[:, None] + demand_normalized - same_bin_penalty - overlap_penalty
    
    threshold = np.percentile(heuristics, 90)
    heuristics[heuristics < threshold] = 0
    
    return heuristics


def heuristics_human(demand: np.ndarray, capacity: int) -> np.ndarray:
    return np.tile(demand/demand.max(), (demand.shape[0], 1))

def solve(inst: BPPInstance, heuristics):
    heu = heuristics(inst.demands.copy(), inst.capacity) # normalized in ACO
    assert tuple(heu.shape) == (inst.n, inst.n)
    assert 0 < heu.max() < np.inf
    aco = ACO(inst.demands, heu.astype(float), capacity = inst.capacity, n_ants=N_ANTS, greedy=False)

    results = []
    for i in range(len(N_ITERATIONS)):
        if i == 0:
            obj, _ = aco.run(N_ITERATIONS[i])
        else:
            obj, _ = aco.run(N_ITERATIONS[i] - N_ITERATIONS[i-1])
        results.append(obj)
    return results

In [14]:
for problem_size in [120]:
    dataset_path = f"dataset/val{problem_size}_dataset.npz"
    dataset = load_dataset(dataset_path)
    n_instances = dataset[0].n
    logging.info(f"[*] Evaluating {dataset_path}")

    objs = []
    for i, instance in enumerate(dataset):
        obj = solve(instance, heuristics_human) # expert-defined heuristics
        objs.append(obj)
    
    # Average objective value for all instances
    mean_obj = np.mean(objs, axis=0)
    for i, obj in enumerate(mean_obj):
        print(f"[*] Average for {problem_size}, {N_ITERATIONS[i]} iterations: {obj}")
    print()

[*] Average for 120, 1 iterations: 51.796875
[*] Average for 120, 10 iterations: 51.171875
[*] Average for 120, 30 iterations: 50.78125
[*] Average for 120, 50 iterations: 50.484375
[*] Average for 120, 80 iterations: 50.203125
[*] Average for 120, 100 iterations: 50.125



In [15]:
for problem_size in [120]:
    dataset_path = f"dataset/val{problem_size}_dataset.npz"
    dataset = load_dataset(dataset_path)
    n_instances = dataset[0].n
    logging.info(f"[*] Evaluating {dataset_path}")

    objs = []
    for i, instance in enumerate(dataset):
        obj = solve(instance, heuristics_reevo) # ReEvo-generated heuristics
        objs.append(obj)
    
    # Average objective value for all instances
    mean_obj = np.mean(objs, axis=0)
    for i, obj in enumerate(mean_obj):
        print(f"[*] Average for {problem_size}, {N_ITERATIONS[i]} iterations: {obj}")
    print()

[*] Average for 120, 1 iterations: 49.890625
[*] Average for 120, 10 iterations: 49.640625
[*] Average for 120, 30 iterations: 49.484375
[*] Average for 120, 50 iterations: 49.421875
[*] Average for 120, 80 iterations: 49.375
[*] Average for 120, 100 iterations: 49.375

